In [ ]:
# ---------------------------------------------------------
# 1. Data Loading
# Load two Excel files: one with employee turnover data (desligamento_df)
# and another with employee survey scores (notas_df).
# Print the column names of both dataframes to verify they loaded correctly.
# ---------------------------------------------------------
desligamento_df = pd.read_excel('C:\\Users\\FabianaKuhlmann\\Downloads\\desligamentos.xlsx')
notas_df = pd.read_excel('C:\\Users\\FabianaKuhlmann\\Downloads\\Pasta3.xlsx')

print(desligamento_df.columns)
print(notas_df.columns)


# ---------------------------------------------------------
# 2. Data Merging
# Merge the turnover and survey dataframes on the 'Franquia' column (franchise).
# This creates a combined dataset for analysis.
# ---------------------------------------------------------
base_merged = pd.merge(desligamento_df, notas_df, on='Franquia')


# ---------------------------------------------------------
# 3. Correlation Analysis
# Compute a correlation matrix for numeric variables to analyze relationships
# between survey scores and different types of turnover.
# ---------------------------------------------------------
correlation_matrix = base_merged.corr(numeric_only=True)


# ---------------------------------------------------------
# 4. Visualization of Correlation
# Create a heatmap to visualize the correlations between survey scores
# and turnover rates (general, involuntary, voluntary).
# ---------------------------------------------------------
plt.figure(figsize=(10, 6))
sns.heatmap(
    correlation_matrix[['%Desligamento (Geral)', 
                        '%Desligamento (involuntário)', 
                        '%Desligamento (voluntário)']].dropna().T,
    annot=True, cmap='coolwarm', center=0
)
plt.title('Correlation between survey scores and turnover types')
plt.show()


# ---------------------------------------------------------
# 5. Regression Analysis with R²
# For each survey column, run a simple linear regression (OLS)
# with voluntary turnover as the dependent variable and the survey score
# as the independent variable. Print the R² for each model.
# ---------------------------------------------------------
colunas_notas = [...]  # Replace with the list of survey score columns
tipo_desligamento = '%Desligamento (voluntário)'  # Target variable (voluntary turnover)

for coluna in colunas_notas:
    X = base_merged[coluna]
    y = base_merged[tipo_desligamento]

    # Add a constant term for the intercept and fit the model
    model = sm.OLS(y, sm.add_constant(X)).fit()
    r2 = model.rsquared
    print(f"{coluna}: R² = {r2:.3f}")
    # Optionally plot regression line and scatter points here...


# ---------------------------------------------------------
# 6. ANOVA Testing
# Perform an ANOVA test to evaluate whether there are statistically
# significant differences in voluntary turnover rates based on
# 'Remuneração e Benefícios' (Compensation & Benefits) survey scores.
# ---------------------------------------------------------
anova_model = smf.ols(f'Q(\"{tipo_desligamento}\") ~ Q(\"Remuneração e Benefícios\")', data=base_merged).fit()
anova_result = sm.stats.anova_lm(anova_model, typ=2)

print("\n--- ANOVA ---")
print(anova_result)


# ---------------------------------------------------------
# 7. Decision Tree Modeling
# Train a Decision Tree Regressor to predict voluntary turnover based on
# all survey scores. Apply the tree to the dataset to assign each observation
# to a final node, and plot the tree structure.
# ---------------------------------------------------------
X_tree = base_merged[colunas_notas]
y_tree = base_merged[tipo_desligamento]

tree = DecisionTreeRegressor(max_depth=3, random_state=42)
tree.fit(X_tree, y_tree)

base_merged['no_final'] = tree.apply(X_tree)  # Final node assigned to each row

plt.figure(figsize=(15, 6))
plot_tree(tree, feature_names=colunas_notas, filled=True, rounded=True)
plt.title("Decision Tree - Predictors of Voluntary Turnover")
plt.show()


# ---------------------------------------------------------
# 8. Multiple Linear Regression
# Fit a multiple linear regression model using all survey score variables
# as predictors and voluntary turnover as the dependent variable.
# Print the summary to inspect coefficients, R², and statistical significance.
# ---------------------------------------------------------
X_multi = base_merged[colunas_notas]
X_multi = sm.add_constant(X_multi)
y_multi = base_merged[tipo_desligamento]

modelo_multipla = sm.OLS(y_multi, X_multi).fit()
print("\n--- Multiple Linear Regression ---")
print(modelo_multipla.summary())


# ---------------------------------------------------------
# 9. Variable Importance from Decision Tree
# Extract the importance of each survey variable from the trained decision tree,
# create a DataFrame sorted by importance, and plot the values in a bar chart.
# ---------------------------------------------------------
importances = tree.feature_importances_
importancia_df = pd.DataFrame({
    'Variable': colunas_notas,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(importancia_df)

sns.barplot(x='Importance', y='Variable', data=importancia_df)
plt.title('Variable Importance - Decision Tree')
plt.show()
